#### 一. Hotspot架构
* JIT在java程序运行中, 针对底层操作系统, 生成高表现力的本地机器代码
* JVM在随着不断演进, 持续迭代java运行时, 而且有多线程的垃圾收集器  
* JVM的主要组建包括:  
    * classloader
    * 运行时数据区
    * 执行引擎   
    
<img src="img/jvm1.png" width="60%" height="60%" align="left"/>  

* JVM的3个关键组件:  
    * 堆
    * JIT编辑器
    * 垃圾回收器     
  关键组件在下图用紫色标出, 调试java程序性能时, 通常使用'堆大小'和'选择哪个垃圾回收器'为主.   
  虽然JIT也关系着程序性能, 单着只能在下个版本的jvm中给出优化
  
  
 <img src="img/jvm2.png" width="60%" height="60%" align="left"/>  
 
 

#### 二. Performance Basics  
* 性能优化的两个目标:  
    * 响应能力 (responsiveness)  
    响应能力表示程序作出响应所消耗的时间,  
    对于关注响应能力的程序, 长的停顿时间是不能被接收的,  典型的例子包括: 
        * 桌面UI对时间的响应时间
        * WebSite返回页面的时间
        * 数据库查询返回数据的时间  
    
    * 吞吐量 (throughput)  
    吞吐量表示一段时间内完成的任务数量  
    对于关注吞吐量的程序, 长时间的停顿可以被接受, 因为这些程序只关注长期表现, 不考虑短期内的响应, 典型例子包括:  
        * 给定时间内完成的事务数量  
        * 1小时内, 批处理程序完成的job数
        * 1小时内, 数据库完成的query数量
        
* 什么是垃圾自动回收  
gc程序一直在监视着heap(堆内存), 并且把正在使用或被引用的对象, 和不再使用或不被引用的对象区分开来.   
一个正在使用或被引用的对象, 说明程序中存在指向该对象的指针; 而不再使用或不被引用的对象说明不存在指向该对象的指针  
所以, 对于不再使用或不被引用的对象占用的内存, 可以被回收. 下面展示这个步骤        

#### 三. 垃圾回收的步骤  


* Parallel GC
    * Parallel GC是多线程的年轻带收集器. 一个n核心的主机上默认使用n个线程进行gc  
      gc线程数控制: `-XX:ParallelGCThreads=<desired number>`
    * 2核心以上的主机使用parallel gc才能有更好的结果
    * 使用场景:  
      Parallel collector也叫做高吞吐收集器, 适用于收集大内存并且允许暂停时间稍长的应用. 如批处理任务,数据库query等
    * `-XX:+UseParallelGC`  
      该参数在青年带上使用多线程收集器, 在老年代上使用单线程收集器
    * `-XX:+UseParallelOldGC`  
      该参数在年轻带和年老带都使用多线程收集器,且在年老带使用压缩(compact)   
      HotSpot虚拟机仅仅在年老带使用压缩清除(compact), 在年轻带使用复制清除(copy), 因此年轻带不需要压缩

* CMS GC  
    * CMS收集器, 全称"Concurrent Mark Sweep (CMS) Collector", 是一种运行在老年代的低暂停,高响应收集器. 通过让垃圾收集线程和应用线程并行的方式最小化暂停时间
    * 这种并发的高响应收集器, 不会把已存在的对象进行copy或compact, 因此会在内存中留下缝隙, 如果这种方式是一个问题的话就要分配更大的老年代  
    * CMS算法在年轻带使用和Parallel GC相同的收集算法  
    * CMS高响应收集器可用在桌面UI,WebSite, 数据库查询响应等场景  
    * 使用CMS: `-XX:+UseConcMarkSweepGC`  
      指定收集线程个数: `-XX:ParallelCMSThreads=<n>`

* G1收集器
    * 在java1.7中被设计出来, 旨在替代CMS收集器, 他同样是高响应,多线程,与应用并行的收集器

#### 四. 默认收集器  
* `java -XX:+PrintCommandLineFlags -version ` : 查看当前gc是哪种
* Default garbage collectors:
    * Java 7 - Parallel GC
    * Java 8 - Parallel GC
    * Java 9 - G1 GC
    * Java 10 - G1 GC